In [1]:
import torch
from torchmetrics.functional.multimodal import clip_score
score = clip_score(torch.randint(255, (3, 1024, 1024)), "a photo of a cat", "/data/chx/clip-vit-large-patch14")
score.detach()

tensor(21.6790)

In [ ]:
import os
import torch
from PIL import Image
from torchvision import transforms
from torchmetrics.functional.multimodal import clip_score
from functools import partial

from transformers import CLIPImageProcessor, CLIPModel, CLIPTokenizer

# 定义CLIP评分函数
clip_score_fn = partial(clip_score, model_name_or_path="/data/chx/clip-vit-large-patch14")

# 加载图片并进行预处理（保持原图尺寸）
def load_and_preprocess_image(image_path):
    image = Image.open(image_path).convert("RGB")
    return image


# 计算CLIP分数
def calculate_clip_score(images, prompts):
    images_tensor = torch.stack([torch.tensor(image['pixel_values'][0]) for image in images])
    clip_score_value = clip_score_fn(images_tensor, prompts).detach()
    return round(float(clip_score_value), 4)

# 图片路径列表
image_paths = [
    "/home/chx/mySrc/diffusers/output/image.webp"
]

# 对应的文本提示
prompts = [
    "a tiny astronaut hatching from an egg on the moon"
]

# 加载并预处理所有图片
images = [load_and_preprocess_image(image_path) for image_path in image_paths]

# 计算并打印CLIP分数
sd_clip_score = calculate_clip_score(images, prompts)
print(f"CLIP score: {sd_clip_score}")

TypeError: 'Image' object is not subscriptable

In [1]:
import os
import torch
from PIL import Image
from torchvision import transforms
from torchmetrics.functional.multimodal import clip_score
from functools import partial
import pandas as pd
from transformers import CLIPImageProcessor, CLIPModel, CLIPTokenizer
from tqdm import tqdm

# 设置环境变量
os.environ['MKL_SERVICE_FORCE_INTEL'] = '1'
os.environ['MKL_THREADING_LAYER'] = 'Intel'
os.environ["CUDA_VISIBLE_DEVICES"] = '2'

# 定义CLIP评分函数
clip_score_fn = partial(clip_score, model_name_or_path="/data/chx/clip-vit-large-patch14")

# 加载图片并进行预处理（保持原图尺寸）
def load_and_preprocess_image(image_path):
    preprocess = CLIPImageProcessor.from_pretrained("/data/chx/clip-vit-large-patch14")
    image = Image.open(image_path).convert("RGB")
    image = preprocess(image, return_tensors="pt", do_rescale=False)
    image['pixel_values'] = torch.clamp(image['pixel_values'], 0, 1)  # 归一化处理
    return image

# 计算CLIP分数
def calculate_clip_score(images, prompts):
    images_tensor = torch.stack([image['pixel_values'][0] for image in images])
    clip_score_value = clip_score_fn(images_tensor, prompts).detach()
    return round(float(clip_score_value), 4)

# 批量计算图片的CLIP分数并存入Excel
def batch_calculate_clip_score(excel_path, output_excel_path):
    # 读取Excel文件
    data = pd.read_excel(excel_path)
    
    # 创建一个新的DataFrame用于存储结果
    results = pd.DataFrame(columns=['Image Path', 'Source Prompt', 'Target Prompt', 'CLIP Score'])
    
    # 创建一个字典用于跟踪每种编辑类别的行序号
    class_row_counters = {}

    # 遍历Excel表中的每一行，使用tqdm显示进度条
    for index, row in tqdm(data.iterrows(), total=len(data)):
        # 提取信息
        edit_class = row['Edit Class']
        source_prompt = row['Source Prompt']
        target_prompt = row['Target Prompt']
        
        # 如果是新出现的编辑类别，初始化其计数
        if edit_class not in class_row_counters:
            class_row_counters[edit_class] = 1
        else:
            class_row_counters[edit_class] += 1
        
        # 根据编辑类别和类内的行序号生成图片路径
        class_row_number = class_row_counters[edit_class]
        source_img_path = f"/data/chx/EditEval_v1/Dataset/{edit_class}/{class_row_number}.jpg"
        
        # 加载并预处理图片
        image = load_and_preprocess_image(source_img_path)
        
        # 计算CLIP分数
        clip_score_value = calculate_clip_score([image], [source_prompt])
        
        # 将结果添加到DataFrame
        new_row = pd.DataFrame({'Image Path': [source_img_path], 'Source Prompt': [source_prompt], 'Target Prompt': [target_prompt], 'CLIP Score': [clip_score_value]})
        if not new_row.empty:
            results = pd.concat([results, new_row], ignore_index=True)
        
        # 输出当前文件名、Prompt、CLIP分数
        print(f"文件名: {source_img_path}, Source Prompt: {source_prompt}, Target Prompt: {target_prompt}, CLIP分数: {clip_score_value}")
    
    # 将结果写入新的Excel文件
    results.to_excel(output_excel_path, index=False)

if __name__ == "__main__":
    # 设置参数
    excel_path = "/data/chx/EditEval_v1/Dataset/editing_prompts_collection.xlsx"
    output_excel_path = "/data/chx/EditEval_v1/output_clip_scores.xlsx"

    # 执行批量计算
    batch_calculate_clip_score(excel_path, output_excel_path)

  0%|          | 0/50 [00:00<?, ?it/s]It looks like you are trying to rescale already rescaled images. If the input images have pixel values between 0 and 1, set `do_rescale=False` to avoid rescaling them again.
/tmp/ipykernel_834889/3008020916.py:70: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results = pd.concat([results, new_row], ignore_index=True)
  2%|▏         | 1/50 [00:07<06:13,  7.63s/it]

文件名: /data/chx/EditEval_v1/Dataset/object_addition/1.jpg, Source Prompt: A deserted highway near snow mountains under a partly cloudy sky, Target Prompt: A red car on a deserted highway near snow mountains under a partly cloudy sky, CLIP分数: 14.9991


  4%|▍         | 2/50 [00:15<06:25,  8.04s/it]

文件名: /data/chx/EditEval_v1/Dataset/object_addition/2.jpg, Source Prompt: A tranquil lake surrounded by dense forests and mountains under a clear blue sky, Target Prompt: A canoe on the tranquil lake surrounded by dense forests and mountains under a clear blue sky, CLIP分数: 15.001


  6%|▌         | 3/50 [00:24<06:20,  8.09s/it]

...

文件名: /data/chx/EditEval_v1/Dataset/action_change/2.jpg, Source Prompt: A polar bear standing beside the sea, Target Prompt: A polar bear raising its hand, CLIP分数: 17.3885


 98%|█████████▊| 49/50 [06:23<00:07,  7.60s/it]

文件名: /data/chx/EditEval_v1/Dataset/action_change/3.jpg, Source Prompt: A dog standing on the ground, Target Prompt: A dog jumping on the ground, CLIP分数: 19.4331


100%|██████████| 50/50 [06:31<00:00,  7.83s/it]

文件名: /data/chx/EditEval_v1/Dataset/action_change/4.jpg, Source Prompt: A bird on the tree, Target Prompt: A bird is flying over the tree, CLIP分数: 16.8844
